In [1]:
%%capture
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip -q install wikipedia
!pip -q install docarray
!pip -q install --upgrade protobuf google.protobuf

In [1]:
# Before proceeding with the next cells restart the kernel by clicking the refresh icon on the top toolbar.

In [2]:
import os
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
from google.protobuf.empty_pb2 import Empty

key_name = !gcloud services api-keys list --filter="gemini-api-key" --format="value(name)"
key_name = key_name[0]

api_key = !gcloud services api-keys get-key-string $key_name --location="us-central1" --format="value(keyString)"
api_key = api_key[0]

os.environ["GOOGLE_API_KEY"] = api_key

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [3]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
      

# Using Gemini directly with Python SDK

In [4]:
# generate text
prompt = 'Who are you and what can you do?'

model = genai.GenerativeModel('gemini-pro')

response = model.generate_content(prompt)

Markdown(response.candidates[0].content.parts[0].text)

I am Gemini, a multimodal AI model, developed by Google. I am designed to provide information and assist with a wide range of language-based tasks, such as answering questions, generating text, translating languages, and writing different kinds of creative content.

Here are some of my capabilities:

**Information Retrieval:**
* Answering factual questions on a diverse range of topics
* Providing summaries of complex information
* Translating text between over 100 languages

**Text Generation:**
* Generating creative text, such as stories, poems, and songs
* Writing different types of content, including articles, blog posts, and marketing copy
* Summarizing long pieces of text

**Language Understanding:**
* Understanding the meaning and context of text
* Identifying and extracting key information
* Recognizing and responding to different types of user intent

**Communication:**
* Engaging in natural language conversations
* Answering follow-up questions
* Providing clear and concise explanations

**Other Capabilities:**
* Code generation
* Math problem solving
* Fact-checking
* Sentiment analysis

I am constantly learning and improving, and my capabilities are expanding all the time. I am not yet able to perform all tasks perfectly, but I am always striving to provide the best possible assistance.

Please let me know if you have any other questions.

# Using Gemini with LangChain

## Basic LLM Chain

In [5]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)


result = llm.invoke("What is a LLM?")

Markdown(result.content)

LLM stands for Large Language Model. It is a type of artificial intelligence (AI) that is trained on a massive dataset of text and code. LLMs are able to understand and generate human-like text, translate languages, write different kinds of creative content, and even write different kinds of code. They are also able to answer questions and perform other tasks that require a deep understanding of language.

LLMs are still under development, but they have already shown great promise for a wide range of applications, including customer service, education, and entertainment. As LLMs continue to improve, they are likely to play an increasingly important role in our lives.

Here are some of the things that LLMs are capable of:
* **Understanding and generating human-like text:** LLMs can understand the meaning of text and generate new text that is fluent, coherent, and informative. This makes them ideal for tasks such as writing articles, stories, and marketing copy.
* **Translating languages:** LLMs can translate text between different languages with a high degree of accuracy. This makes them ideal for tasks such as customer service, travel, and education.
* **Writing different kinds of creative content:** LLMs can write different kinds of creative content, such as poems, songs, and screenplays. This makes them ideal for tasks such as entertainment, education, and marketing.
* **Answering questions:** LLMs can answer questions on a wide range of topics, including history, science, and current events. This makes them ideal for tasks such as customer service, education, and research.
* **Performing other tasks that require a deep understanding of language:** LLMs can perform a variety of other tasks that require a deep understanding of language, such as summarizing text, identifying sentiment, and detecting plagiarism. This makes them ideal for tasks such as research, data analysis, and content moderation.

LLMs are a powerful tool that can be used for a wide range of applications. As they continue to improve, they are likely to play an increasingly important role in our lives.

In [6]:
for chunk in llm.stream("Write a haiku about LLMs."):
    print(chunk.content)
    print("---")

Words dance in the void,
AI's boundless language stream,
Thoughts
---
 take flight anew.
---


## Basic Multi Chain

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

In [8]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

output_parser = StrOutputParser()

In [9]:
chain = prompt | model | output_parser

In [10]:
chain.invoke({"topic": "machine learning"})

'Why did the machine learning algorithm get a speeding ticket?\n\nBecause it was caught accelerating too fast.'

## A more complicated Chain - RAG

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain.document_loaders import WikipediaLoader

# use Wikipedia loader to create some docs to use..
docs = WikipediaLoader(query="Machine Learning", load_max_docs=10).load()
docs += WikipediaLoader(query="Deep Learning", load_max_docs=10).load() 
docs += WikipediaLoader(query="Neural Networks", load_max_docs=10).load() 

# Take a look at a single document
docs[0]

In [ ]:
vectorstore = DocArrayInMemorySearch.from_documents(
    docs,
    embedding=embeddings # passing in the model to embed documents..
)

retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents("what is machine learning?")

In [ ]:
retriever.get_relevant_documents("what is gemini pro?")

In [ ]:
template = """Answer the question a a full sentence, based only on the following context:
{context}

Return you answer in three back ticks

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
retriever.get_relevant_documents("What is a graident boosted tree?")

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "What is machine learning?"})

In [ ]:
chain.invoke({"question": "When was the transformer invented?"})